In [ ]:
import numpy as np
import pandas as pd
import h5py
import cv2
from tqdm import tqdm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn import model_selection

In [ ]:
# each image is represented by a 30 x 30 matrix
x = np.array(h5py.File('./datasets/train_cleaned_tenK.hdf5', 'r')['train_preprocessed'])
y = np.array(pd.read_csv('./datasets/original/train_labels.csv', usecols=['Category']))
y = [label[0] for label in y]
# final_x  = np.array(h5py.File('test_vectors_tenK.hdf5', 'r')['test_preprocessed'])

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.4, shuffle=False)
test_x, valid_x, test_y, valid_y = train_test_split(test_x, test_y, test_size=0.5, shuffle=False)

In [ ]:
# KNN using max-pooling
# Each image is represented by a 30 x 30 matrix

def max_pooling(img, side_len):
  slice_x = int(30/side_len)
  slice_y = slice_x
  res = np.zeros((slice_x, slice_y))
  for i in range(0, slice_x):
      for j in range(0, slice_y):
          max = 0
          for p in range(i*side_len, (i+1)*side_len):
              for q in range(j*side_len, (j+1)*side_len):
                  val = img[p][q]
                  if val>max: 
                      max = val
          res[i][j] = max
  return res
    
def flatten(l):
  return [item for sublist in l for item in sublist]


num_neighbor = 1000      

In [ ]:
def find_contours(img_gray):
  """
  Finding contours for the
  given greyscale image

  @param
      :type np.array img_gray - greyscale version of the image

  @return
      :rtype np.array - the image
      :rtype np.array - contours found for the image

  """
  ret, thresh = cv2.threshold(img_gray, 127, 255, 0)
  thresh = thresh.astype('uint8')
  image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  return contours

In [ ]:
from operator import itemgetter
  
def find_nearest_neighbors(k, image):
  cnt1 = find_contours(image)[0]
  dist = []
  for i in range(0, len(train_x)):
    img = train_x[i]
    cnt2 = find_contours(img)[0]
    ret = cv2.matchShapes(cnt1, cnt2, 1, 0.0)
    dist.append([ret, i])
   
  sorted_dist = sorted(dist, key=itemgetter(0))[:k]
  return sorted_dist


def vote(dist):
  dict = {}
  for (ret, i) in dist:
    label = train_y[i]
    if label not in dict:
      dict[label] = 1
    else:
      dict[label] += 1
  
  max = 0
  max_label = 0
  for label in dict:
    if dict[label]>max:
      max = dict[label]
      max_label = label
      
  return max_label

In [ ]:
# modelling with max-pooling
side_len = 3
processed_train_x = []
for image in train_x:
  pooled_image = max_pooling(image, side_len)
  flattened_image = flatten(pooled_image)
  processed_train_x.append(flattened_image)

flattened_test_x = []
for image in test_x:
  pooled_image = max_pooling(image, side_len)
  flattened_image = flatten(pooled_image)
  flattened_test_x.append(flattened_image)

param_grid_knn = {'n_neighbors': [10, 11, 12, 13, 14, 15, 20, 25]} 
clf = KNeighborsClassifier()
clf = model_selection.GridSearchCV(clf, param_grid_knn, cv=3, scoring='f1_micro', verbose=2)
clf.fit(processed_train_x, train_y)
clf.score(flattened_test_x, test_y)         

In [ ]:
num_neighbors = 100

predictions = []
for image in tqdm(test_x):  
  neighbors = find_nearest_neighbors(num_neighbors, image)
  predict_y = vote(neighbors)
  predictions.append(predict_y)

f1_score(test_y, predictions, average='weighted')  

In [ ]:
clf.best_params_

In [ ]:
print(neighbors)

In [ ]:
print(predictions)